In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
current = os.getcwd()
xls_data_files = os.path.join(current,  "xls_data_files")
pickle_data_files = os.path.join(current,  "pickle_files")
processed_data_files = os.path.join(current,  "processed_files")


In [ ]:


def open_tinytag_xls(file, dir_):
    
    # parse the XML file
    tree = ET.parse(os.path.join(dir_, file))

    # get the root of the XML document
    root = tree.getroot()

    # Define the namespace
    ns = {'ss': 'urn:schemas-microsoft-com:office:spreadsheet'}

    # Access 'Sheet1'
    sheet1 = root.find("ss:Worksheet[@ss:Name='Sheet1']", namespaces=ns)

    # To access rows in 'Sheet1'
    rows = sheet1.findall('.//ss:Row', namespaces=ns)

    data = []

    for row in rows:
        rowData = []
        cells = row.findall('.//ss:Cell', namespaces=ns)
        for cell in cells:
            dataElem = cell.find('.//ss:Data', namespaces=ns)
            rowData.append(dataElem.text if dataElem is not None else None)
        data.append(rowData)
        
    if file[0] == 'R':
        df = pd.DataFrame(data, columns=["index", 'date', "temp_air", "relative_humidity", 'dew_point']).iloc[5:][['date', "temp_air", "relative_humidity"]]
        df['date'] = pd.to_datetime(df['date'])
        df['temp_air'] = df['temp_air'].astype(float).round(1)
        df['relative_humidity'] = df['relative_humidity'].astype(float).round(0)
        return df
    else:
        df = pd.DataFrame(data, columns=["index", 'date', "temp_air", "temp_pulp"]).iloc[5:][['date', "temp_air", "temp_pulp"]]
        df['date'] = pd.to_datetime(df['date'])
        df['temp_air'] = df['temp_air'].astype(float).round(1)
        df['temp_pulp'] = df['temp_pulp'].astype(float).round(1)
        return df

In [ ]:
for file in os.listdir(xls_data_files):
    df = open_tinytag_xls(file, xls_data_files)
    df.to_pickle(os.path.join(pickle_data_files, file.split('.')[0] + '.pickle'), compression='gzip')

In [ ]:
log_pos = pd.read_excel('logger_positions.xlsx')
log_pos['id'] = log_pos['treatment'] + "_" + log_pos['location_c'] + "_" + log_pos['location_p']
log_pos = log_pos.set_index('logger')['id']
log_pos = log_pos.to_dict()

In [ ]:
start = pd.Timestamp(2023, 4, 26, 23, 45)
end = pd.Timestamp(2023, 5, 29, 9, 0)

In [ ]:
for file in os.listdir(pickle_data_files):
    name = file.split('.')[0]
    df = pd.read_pickle(os.path.join(pickle_data_files, file), compression='gzip')
    df = df[(df['date'] >= start) & (df['date'] <= end)]
    df['hours'] = (df['date'] - df['date'].iloc[0]).dt.total_seconds() / 3600.0
    df['hours'] = df['hours'].astype(float).round(2)
    df['days'] = (df['date'] - df['date'].iloc[0]).dt.total_seconds() / (3600.0 * 24)
    df['days'] = df['days'].astype(float).round(5)
    # 
    
    if log_pos[name][:5] == 'CN_M_':
        try: df['offset_temp_air'] = df['temp_air']
        except: print(log_pos[name])
        try: df['offset_temp_pulp'] = df['temp_pulp']
        except: print(log_pos[name])
        
    if log_pos[name][:5] == 'CN_B_':
        try: df['offset_temp_air'] = df['temp_air'] - 1
        except: print(log_pos[name])
        try: df['offset_temp_pulp'] = df['temp_pulp'] - 1
        except: print(log_pos[name])
        
    if log_pos[name][:5] == 'U2_M_':
        try: df['offset_temp_air'] = df['temp_air']
        except: print(log_pos[name])
        try: df['offset_temp_pulp'] = df['temp_pulp']
        except: print(log_pos[name])
        
    if log_pos[name][:5] == 'U2_B_':
        try: df['offset_temp_air'] = df['temp_air'] - 1
        except: pass
        try: df['offset_temp_pulp'] = df['temp_pulp'] - 1 
        except: pass
        
    try:
        df.to_pickle(os.path.join(processed_data_files, log_pos[name] + '.pickle'), compression='gzip')
    except: print(name)